# US Water Quality Data Portal
https://www.waterqualitydata.us/portal/

In [1]:
#Imports
import pandas as pd

## Bokeh map components
from bokeh.models import ColumnDataSource,WMTSTileSource
from bokeh.plotting import figure, show
from bokeh.tile_providers import *
from bokeh.io import output_notebook

#Display Bokeh in notebook
output_notebook()

Loading BokehJS ...

## Get data
These are files generated `from USWaterData-Scrape.ipynb`

In [6]:
#Convert to dataframe
dfSites = pd.read_csv("../../Data/USWQP/DurhamSites.csv")
dfResults = pd.read_csv("../../Data/USWQP/DurhamResults.csv",low_memory=False)
dfSamples = pd.read_csv("../../Data/USWQP/DurhamSamples.csv",low_memory=False)
dfActivity = pd.read_csv("../../Data/USWQP/DurhamActivity.csv")

In [7]:
#Display the first record
df = dfSites.dropna(how='all',axis=1)
df.columns

Index(['OrganizationIdentifier', 'OrganizationFormalName',
       'MonitoringLocationIdentifier', 'MonitoringLocationName',
       'MonitoringLocationTypeName', 'MonitoringLocationDescriptionText',
       'HUCEightDigitCode', 'DrainageAreaMeasure/MeasureValue',
       'DrainageAreaMeasure/MeasureUnitCode',
       'ContributingDrainageAreaMeasure/MeasureValue',
       'ContributingDrainageAreaMeasure/MeasureUnitCode', 'LatitudeMeasure',
       'LongitudeMeasure', 'SourceMapScaleNumeric',
       'HorizontalAccuracyMeasure/MeasureValue',
       'HorizontalAccuracyMeasure/MeasureUnitCode',
       'HorizontalCollectionMethodName',
       'HorizontalCoordinateReferenceSystemDatumName',
       'VerticalMeasure/MeasureValue', 'VerticalMeasure/MeasureUnitCode',
       'VerticalAccuracyMeasure/MeasureValue',
       'VerticalAccuracyMeasure/MeasureUnitCode',
       'VerticalCollectionMethodName',
       'VerticalCoordinateReferenceSystemDatumName', 'CountryCode',
       'StateCode', 'CountyCode',

In [8]:
df.OrganizationIdentifier.unique()

array(['USGS-NC', '21NC01WQ', '21NC02WQ', '21NC03WQ', '21NCCOALITIONS',
       '21NCMONITORING', 'GLEON', 'NARSTEST', 'NARS_WQX'], dtype=object)

## Mapping

In [9]:
#Function to convert WGS84 points to Web Mercator
def wgs84_to_web_mercator(df, lon="lon", lat="lat"):
    """Converts decimal longitude/latitude to Web Mercator format"""
    import numpy as np
    k = 6378137
    df["x"] = df[lon] * (k * np.pi/180.0)
    df["y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k
    return df

In [10]:
#Generate colors
from bokeh.palettes import Category20
import itertools

def color_gen():
    yield from itertools.cycle(Category20[20])
color = color_gen()

In [11]:
#Add web mercator coordinates
df = wgs84_to_web_mercator(df,lon='LongitudeMeasure',lat='LatitudeMeasure')

In [12]:
#Exaxmine the different types of Monitoring Locations
df.MonitoringLocationIdentifier.unique()

array(['USGS-02085039', 'USGS-020850391920', 'USGS-020850391940',
       'USGS-020850391960', 'USGS-0208503920', 'USGS-0208503945',
       'USGS-0208503990', 'USGS-0208505878', 'USGS-0208505880',
       'USGS-02085064', 'USGS-02085066', 'USGS-02085067', 'USGS-02085070',
       'USGS-02085079', 'USGS-0208521324', 'USGS-02085220',
       'USGS-0208524088', 'USGS-0208524090', 'USGS-0208524170',
       'USGS-0208524845', 'USGS-0208524850', 'USGS-0208524930',
       'USGS-0208524950', 'USGS-0208524975', 'USGS-0208525095',
       'USGS-0208525105', 'USGS-02085262', 'USGS-0208527100',
       'USGS-0208527550', 'USGS-02085284', 'USGS-0208528420',
       'USGS-02085477', 'USGS-02085500', 'USGS-0208590000',
       'USGS-02086000', 'USGS-0208610150', 'USGS-0208629180',
       'USGS-02086300', 'USGS-02086490', 'USGS-02086500',
       'USGS-0208650112', 'USGS-0208651170', 'USGS-02086720',
       'USGS-0208675009', 'USGS-0208675010', 'USGS-02086760',
       'USGS-02086770', 'USGS-02086790', 'USGS-02

In [13]:
#Set the Bokeh CDS
sources = {}
for type in df.MonitoringLocationTypeName.unique().tolist():
    sources[type] = ColumnDataSource(df[df.MonitoringLocationTypeName == type])
sources.keys()

dict_keys(['Facility Other', 'Lake', 'Facility: Water-use establishment', 'Facility: Pavement', 'Atmosphere', 'Lake, Reservoir, Impoundment', 'River/Stream', 'Facility: Outfall', 'Facility: Diversion', 'Well', 'Reservoir', 'Stream: Ditch', 'Stream', 'Well: Hyporheic-zone well'])

In [14]:
from bokeh.palettes import Category20
xMin = df.x.min();xMax = df.x.max()
yMin = df.y.min();yMax = df.y.max()
p = figure(x_range=(xMin, xMax), y_range=(yMin, yMax),
           x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(STAMEN_TERRAIN)
for i,type in enumerate(sources.keys()):
    source = sources[type]
    p.circle(x='x',y='y',source=source,color=Category20[16][i])
show(p);